In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

# Define class and funciton for build tree:
1. treeNode (present node of tree).
2. cal_entropy: function calculates entropy.
3. decisionTree: class presents tree.

In [2]:
class treeNode:
    def __init__(self, ids = [], entropy = 0, children = [], split_attribute = None, label = None, order = [], depth = 0):
        self.ids = ids
        self.children = children
        self.entropy = entropy
        self.split_attribute = split_attribute
        self.order = order
        self.depth = depth
    def set_label(self, label):
        self.label = label
    def set_property(self, split_attribute, order):
        self.split_attribute = split_attribute
        self.order = order
    def set_children(children =[]):
        self.children = children

In [3]:
def cal_entropy(freq):
    freq_non_zero = freq[np.array(freq).nonzero()[0]]
    prob = freq_non_zero / float(freq_non_zero.sum())
    
    return -np.sum(prob * np.log(prob))

In [4]:
class decisionTree:
    def __init__(self, root = None, data = None, max_depth = 10, min_split = 2, min_gain = 1e-4):
        self.root = root
        self.data = data
        self.max_depth = max_depth
        self.min_split = min_split
        self.min_gain = min_gain
        self.attributes = None
    def split(self, node):
        ids = node.ids
        node_data = self.data.iloc[ids, :]
        best_attribute = None
        best_gain = 0
        values = []
        best_splits = []
        order = None
        
        for i, att in enumerate(self.attributes):
            splits = []
            values = self.data.iloc[ids, i].unique().tolist()
            if(len(values) == 1):
                continue
            for val in values:
                sub_ids = node_data.index[node_data[att] == val].tolist()
                splits.append(sub_ids)
            if min(map(len, splits)) < self.min_split:
                continue
                
            entropy_att = 0
            for split in splits:
                entropy_att += len(split) * self._entropy(split) / len(ids)

            Information_gain = node.entropy - entropy_att

            if Information_gain < self.min_gain:
                continue
            elif Information_gain > best_gain:
                best_gain = Information_gain
                best_attribute = att
                best_splits = splits
                order = values
        node.set_property(split_attribute = best_attribute, order = order)
        child_node = [treeNode(ids = split, entropy = self._entropy(split), depth = node.depth +1) for split in best_splits]
        return child_node
    
    def train_function(self, data, target):
        self.data = data
        self.attributes = list(data)
        self.target = target
        number_of_data = data.count()[0]
        ids = list(range(number_of_data))
        
        self.root = treeNode(ids = ids, entropy = self._entropy(ids), depth = 0)
        queue = [self.root]
        
        while queue:
            node = queue.pop()
            if node.depth < self.max_depth and node.entropy > self.min_gain:
                node.children = self.split(node)
                if not node.children:
                    self._label(node)
                queue += node.children
            else:
                self._label(node)
                
    def _label(self, node):
        node.set_label(self.target.loc[node.ids].mode().loc[0].values[0])
    def predict_function(self, data):
        self.data = data
        number_of_data = self.data.count()[0]
        labels = [None]*number_of_data
        
        for i in range(number_of_data):
            x = self.data.iloc[i, :]
            node = self.root
            while node.children:
                try:
                    node = node.children[node.order.index(x[node.split_attribute])]
                except:
                    labels[i] = "no"
                    break
            if labels[i] == None:
                labels[i] = node.label
        return labels
    def _entropy(self, ids):
        if(len(ids) == 0):
            return 0
        freq = np.array(self.target.loc[ids].value_counts())
        return cal_entropy(freq)

<h1> Decision Tree with ID3 algorithm </h1>
<blockquotes> The code above is used for set up decision tree. <br>
    In that, each node is defined with entropy, children, attibute of data that used to split points of data at this node.<br>
    Tree has max depth, min_split and min_gain - used to stop the training process attribute, data that stored data for training set.

In [5]:
inputFile = pd.read_csv('train.csv')

In [6]:
X = inputFile.iloc[:, :-1]
Y = inputFile.iloc[:, -1]

# Train test split.
Test set = 0.2 original set.

In [7]:
X_train , X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [8]:
X_train = X_train.reset_index().drop(["index"], axis = 1)
X_test = X_test.reset_index().drop(["index"], axis = 1)
Y_train = Y_train.reset_index().drop(["index"], axis = 1)
Y_test = Y_test.reset_index().drop(["index"], axis = 1)

# Define tree and train.

In [10]:
tree = decisionTree(max_depth = 5, min_split = 2, min_gain = 1e-4)

In [11]:
tree.train_function(X_train, Y_train)

In [12]:
print(tree.root.entropy)

0.6916712647293481


In [13]:
result = tree.predict_function(X_test)

# Calculate accuracy and f1_score for prediction

In [14]:
lb = LabelEncoder().fit(list(np.array(Y_test)))
y_transform = lb.transform(list(np.array(Y_test)))
result_transform = lb.transform(result)

C:\Users\longg\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [16]:
f1 = f1_score(y_transform, result_transform)
acc = accuracy_score(y_transform, result_transform)

In [17]:
print("f1 score: ", f1)
print("acc: ", acc)

f1 score:  0.8001885014137606
acc:  0.8101209135691895
